### 3.1 Обучение модели
Разработайте агента моделирования, который на основе размеченного набора данных обучает модель машинного обучения для классификации транзакций по уровню риска (низкий, средний, высокий) и сложности верификации. Агент использует несколько алгоритмов и сравнивает их по метрикам: точность, полнота, F1-мера, ROC-AUC. Выбирает лучшую модель, тестирует её на отложенной выборке и сохраняет в указанной директории. 

In [1]:
# =========================
# MODULE C / 3.1 — TRAINING AGENT (CLEAN UNIVERSAL TEMPLATE)
# =========================
# Что делает:
# - грузит размеченную таблицу transactions_labeled из SQLite
# - готовит X/y для двух целей:
#   1) risk_level (low/medium/high)
#   2) verification_complexity (simple/medium/hard)
# - пробует несколько моделей, считает метрики, выбирает лучшую по F1_macro
# - сохраняет 2 модели + metrics_31.csv
#
# Почему 2 модели?
# Потому что 2 разные цели (risk и complexity) -> две задачи классификации.

import os
import sqlite3
import numpy as np
import pandas as pd
import joblib

from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.impute import SimpleImputer

from sklearn.metrics import (
    accuracy_score, recall_score, f1_score, roc_auc_score, classification_report
)

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier


# ============================================================
# 0) SETTINGS — обычно меняешь только это на соревнованиях
# ============================================================

DB_PATH = "../db/app.db"                 # <-- поменяй путь, если запускаешь из другой папки
LABELED_TABLE = "transactions_labeled"   # <-- имя таблицы из 2.3

TARGET_RISK = "risk_level"                 # <-- колонка цели №1
TARGET_COMPLEX = "verification_complexity" # <-- колонка цели №2

MODEL_DIR = "../models"                  # <-- куда сохранять
os.makedirs(MODEL_DIR, exist_ok=True)

RANDOM_STATE = 42
TEST_SIZE = 0.2

# Ограничение по строкам (на MacBook Air лучше 100k–300k; на мощном ПК можно 500k–2M)
MAX_TRAIN_ROWS = 200_000   # <-- None если хочешь весь датасет

# ⚠️ Защита от leakage:
# Если метки (risk_level) сделаны из anomaly_score/rule_score/risk_score,
# то эти колонки НЕЛЬЗЯ оставлять в признаках — иначе метрики будут "идеальные", но нечестные.
DROP_COLS = [
    TARGET_RISK,
    TARGET_COMPLEX,

    # --- анти-leakage (по умолчанию включено) ---
    "rule_score",
    "anomaly_score",
    "risk_score",

    # --- часто ещё убирают строковую дату, если она мешает ---
    # "tr_datetime",
]


# ============================================================
# 1) LOAD DATA (SQLite)
# ============================================================

def load_table_sqlite(db_path: str, table: str) -> pd.DataFrame:
    if not os.path.exists(db_path):
        raise FileNotFoundError(f"DB not found: {db_path}")

    con = sqlite3.connect(db_path)
    try:
        df_ = pd.read_sql(f"SELECT * FROM {table}", con)
    finally:
        con.close()

    return df_

df = load_table_sqlite(DB_PATH, LABELED_TABLE)

print("✅ Dataset loaded")
print(f"➡ Rows: {df.shape[0]:,} | Columns: {df.shape[1]}")
print("📌 Columns:", ", ".join(df.columns))
print("\n📄 Preview (first 3 rows):")
display(df.head(3))

# Проверка наличия целей
missing_targets = [c for c in [TARGET_RISK, TARGET_COMPLEX] if c not in df.columns]
if missing_targets:
    raise ValueError(f"❌ Missing target columns: {missing_targets}")
print("\n✅ Target columns found:", [TARGET_RISK, TARGET_COMPLEX])

# Базовая чистка
df = df.replace([np.inf, -np.inf], np.nan)

# Сэмпл (если нужно)
if MAX_TRAIN_ROWS is not None and len(df) > int(MAX_TRAIN_ROWS):
    df = df.sample(n=int(MAX_TRAIN_ROWS), random_state=RANDOM_STATE).reset_index(drop=True)
    print(f"\n⚡ Sampled to {len(df):,} rows for training")
else:
    print("\n⚡ Sampling not required")


# ============================================================
# 2) BUILD X, y
# ============================================================

# Признаки = всё, кроме DROP_COLS
feature_cols = [c for c in df.columns if c not in DROP_COLS]
X = df[feature_cols].copy()

y_risk = df[TARGET_RISK].astype(str)
y_cx = df[TARGET_COMPLEX].astype(str)

# Типы колонок
num_cols = X.select_dtypes(include=["int64", "int32", "float64", "float32"]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

print(f"\n[OK] Features: {len(feature_cols)} | num: {len(num_cols)} | cat: {len(cat_cols)}")

# Train/test split (стратификация по risk)
X_train, X_test, y_risk_train, y_risk_test, y_cx_train, y_cx_test = train_test_split(
    X, y_risk, y_cx,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    stratify=y_risk
)
print(f"[OK] train: {X_train.shape} | test: {X_test.shape}")


# ============================================================
# 3) PREPROCESSING (главный фикс sparse/dense)
# ============================================================

# A) OneHot -> часто sparse -> идеально для линейных моделей
preprocess_onehot = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler(with_mean=False))  # safe with sparse
        ]), num_cols),

        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]), cat_cols),
    ],
    remainder="drop"
)

# B) Ordinal -> dense -> подходит деревьям/HistGB
preprocess_ordinal = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median"))
        ]), num_cols),

        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("ord", OrdinalEncoder(
                handle_unknown="use_encoded_value",
                unknown_value=-1
            ))
        ]), cat_cols),
    ],
    remainder="drop"
)


# ============================================================
# 4) MODELS (быстро + универсально)
# ============================================================

MODELS = {
    "LogReg": ("onehot", LogisticRegression(
        max_iter=2000,
        n_jobs=-1,
        class_weight="balanced"
    )),

    "RandomForest": ("ordinal", RandomForestClassifier(
        n_estimators=250,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        class_weight="balanced_subsample"
    )),

    "HistGB": ("ordinal", HistGradientBoostingClassifier(
        random_state=RANDOM_STATE
    )),
}


# ============================================================
# 5) METRICS
# ============================================================

def eval_multiclass(y_true, y_pred, y_proba=None):
    out = {
        "accuracy": accuracy_score(y_true, y_pred),
        "recall_macro": recall_score(y_true, y_pred, average="macro", zero_division=0),
        "f1_macro": f1_score(y_true, y_pred, average="macro", zero_division=0),
        "roc_auc_ovr": np.nan
    }
    if y_proba is not None:
        try:
            out["roc_auc_ovr"] = roc_auc_score(y_true, y_proba, multi_class="ovr")
        except Exception:
            pass
    return out


def train_and_select(Xtr, ytr, Xte, yte, target_name: str):
    rows = []
    best_name, best_pipe, best_f1 = None, None, -1

    for name, (prep_kind, clf) in MODELS.items():
        preprocess = preprocess_onehot if prep_kind == "onehot" else preprocess_ordinal

        pipe = Pipeline(steps=[
            ("preprocess", preprocess),
            ("model", clf)
        ])

        pipe.fit(Xtr, ytr)
        pred = pipe.predict(Xte)

        proba = None
        if hasattr(pipe.named_steps["model"], "predict_proba"):
            try:
                proba = pipe.predict_proba(Xte)
            except Exception:
                proba = None

        m = eval_multiclass(yte, pred, proba)
        m["model"] = name
        m["target"] = target_name
        rows.append(m)

        print(f"[{target_name}] {name}: f1_macro={m['f1_macro']:.4f} acc={m['accuracy']:.4f}")

        if m["f1_macro"] > best_f1:
            best_f1 = m["f1_macro"]
            best_name = name
            best_pipe = pipe

    res = pd.DataFrame(rows).sort_values("f1_macro", ascending=False).reset_index(drop=True)
    return res, best_name, best_pipe


# ============================================================
# 6) TRAIN for BOTH targets
# ============================================================

print("\n=== TRAIN: risk_level ===")
res_risk, best_risk_name, best_risk_pipe = train_and_select(
    X_train, y_risk_train, X_test, y_risk_test, "risk_level"
)

print("\n=== TRAIN: verification_complexity ===")
res_cx, best_cx_name, best_cx_pipe = train_and_select(
    X_train, y_cx_train, X_test, y_cx_test, "verification_complexity"
)

print("\n=== RESULTS TABLE: risk_level ===")
print(res_risk.to_string(index=False))

print("\n=== RESULTS TABLE: verification_complexity ===")
print(res_cx.to_string(index=False))

print(f"\n✅ Best risk model: {best_risk_name}")
print(f"✅ Best complexity model: {best_cx_name}")


# ============================================================
# 7) REPORTS (чтобы на защите показать)
# ============================================================

print("\n--- BEST RISK MODEL REPORT ---")
risk_pred = best_risk_pipe.predict(X_test)
print(classification_report(y_risk_test, risk_pred, zero_division=0))

print("\n--- BEST COMPLEXITY MODEL REPORT ---")
cx_pred = best_cx_pipe.predict(X_test)
print(classification_report(y_cx_test, cx_pred, zero_division=0))


# ============================================================
# 8) SAVE MODELS + METRICS
# ============================================================

risk_path = os.path.join(MODEL_DIR, "best_model_risk.joblib")
cx_path = os.path.join(MODEL_DIR, "best_model_complexity.joblib")
metrics_path = os.path.join(MODEL_DIR, "metrics_31.csv")

joblib.dump(best_risk_pipe, risk_path)
joblib.dump(best_cx_pipe, cx_path)

pd.concat([res_risk, res_cx], ignore_index=True).to_csv(metrics_path, index=False)

print("\n[SAVED]")
print(" -", risk_path)
print(" -", cx_path)
print(" -", metrics_path)

✅ Dataset loaded
➡ Rows: 2,000,000 | Columns: 12
📌 Columns: customer_id, tr_datetime, mcc_code, tr_type, amount, hour, flow, rule_score, anomaly_score, risk_score, risk_level, verification_complexity

📄 Preview (first 3 rows):


,customer_id,tr_datetime,mcc_code,tr_type,amount,hour,flow,rule_score,anomaly_score,risk_score,risk_level,verification_complexity
0,39026145,0 10:23:26,4814,1030,-2245.92,10,spend,20.0,4.765518,13.906207,low,medium
1,39026145,1 10:19:29,6011,7010,56147.89,10,income,70.0,6.539441,44.615776,medium,simple
2,39026145,1 10:20:56,4829,2330,-56147.89,10,spend,70.0,6.539441,44.615776,medium,simple



✅ Target columns found: ['risk_level', 'verification_complexity']

⚡ Sampled to 200,000 rows for training

[OK] Features: 7 | num: 5 | cat: 2
[OK] train: (160000, 7) | test: (40000, 7)

=== TRAIN: risk_level ===


/Users/iriska/Desktop/transaction_risk_system/venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 2000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=2000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[risk_level] LogReg: f1_macro=0.4521 acc=0.6516
[risk_level] RandomForest: f1_macro=0.8912 acc=0.9555
[risk_level] HistGB: f1_macro=0.8989 acc=0.9569

=== TRAIN: verification_complexity ===
[verification_complexity] LogReg: f1_macro=0.4359 acc=0.7177
[verification_complexity] RandomForest: f1_macro=0.6412 acc=0.9748
[verification_complexity] HistGB: f1_macro=0.6386 acc=0.9709

=== RESULTS TABLE: risk_level ===
 accuracy  recall_macro  f1_macro  roc_auc_ovr        model     target
 0.956900      0.880467  0.898898     0.990947       HistGB risk_level
 0.955475      0.864161  0.891171     0.989360 RandomForest risk_level
 0.651575      0.498787  0.452071     0.769140       LogReg risk_level

=== RESULTS TABLE: verification_complexity ===
 accuracy  recall_macro  f1_macro  roc_auc_ovr        model                  target
 0.974775      0.633847  0.641170     0.971027 RandomForest verification_complexity
 0.970875      0.631654  0.638587     0.705348       HistGB verification_complexity
 0